In [1]:
%gui qt5

In [2]:
import napari
from skimage import data
from pathlib import Path
from dask_image.imread import imread
import dask
from typing import Dict, List, Union, Tuple
import numpy as np
import dask.array as da
import cv2
from threading import Lock
from napari_plugin_engine import napari_hook_implementation

In [3]:
rawdir = Path(r"C:\Data\Zoo_Jwasp_test")
vidpaths = list(rawdir.glob("j*.avi"))
vidpath = vidpaths[0]
vidpath

WindowsPath('C:/Data/Zoo_Jwasp_test/jwasp0.avi')

In [4]:
cap=cv2.VideoCapture(str(vidpath))
cap

<VideoCapture 000001D013444C90>

In [5]:
def get_frame(cap: cv2.VideoCapture, idx: int, *, lock: Lock = Lock()) -> np.ndarray:
    with lock:  # prevent anything else from happening during reading
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        valid, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return frame


def get_height(cap: cv2.VideoCapture) -> int:
    return int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    
def get_width(cap: cv2.VideoCapture) -> int:
    return int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))


def get_nframes(cap: cv2.VideoCapture) -> int:
    return int(cap.get(cv2.CAP_PROP_FRAME_COUNT))


def to_dask_array(cap: cv2.VideoCapture, n_channels = 3, dtype=np.uint8) -> da.Array:
    frame_shape = (get_height(cap), get_width(cap), n_channels)
    n_frames = get_nframes(cap)
    delayeds = [dask.delayed(get_frame)(cap, idx) for idx in range(n_frames)]
    arrays = [da.from_delayed(d, shape=frame_shape, dtype=dtype) for d in delayeds]
    array = da.stack(arrays)
    return array
    

arr = to_dask_array(cap)
arr

,Array,Chunk
Bytes,27.48 GiB,4.45 MiB
Shape,"(6325, 1080, 1440, 3)","(1, 1080, 1440, 3)"
Count,18975 Tasks,6325 Chunks
Type,uint8,numpy.ndarray


In [14]:
arr.ndim

4

In [16]:
arr.dtype

dtype('uint8')

In [15]:
arr.size

29509920000

In [10]:
arr.name = 'filename2.mpg'

TypeError: Cannot set name directly

Name is used to relate the array to the task graph.
It is uncommon to need to change it, but if you do
please set ``._name``

In [7]:
arr[:, :, :, 0]

dask.array<getitem, shape=(6325, 1080, 1440), dtype=uint8, chunksize=(1, 1080, 1440), chunktype=numpy.ndarray>

In [8]:
viewer = napari.view_image(arr[:, :, :, 0])
viewer

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 539.5, 719.5), zoom=0.45125, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 6325.0, 1.0), (0.0, 1080.0, 1.0), (0.0, 1440.0, 1.0)), current_step=(3162, 540, 720), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x1d02a84c130>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ]), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_verti

In [39]:
layer = viewer.layers['arr']
layer

<Image layer 'arr' at 0x23a27da69d0>

In [13]:
%%timeit
arr[10].compute()

110 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
arr[1:3].compute()

In [ ]:
a2[5:10].compute()

In [47]:
da.concatenate(arrs, )

dask.array<concatenate, shape=(1080000, 1440, 3), dtype=uint8, chunksize=(1080, 1440, 3), chunktype=numpy.ndarray>

In [18]:
@napari_hook_implementation
def napari_get_reader(path):
    # remember, path can be a list, so we check it's type first...
    if isinstance(path, str) and any([path.endswith(ext) for ext in [".mp4", ".mov", ".avi"]]):
        cap=cv2.VideoCapture(str(path))
        array = to_dask_array(cap)
        return array
        